## Εργασία 3 ##

Καλωσήρθατε στην τρίτη σας εργασία. Η εργασία αυτή έχει σκοπό να σας βοηθήσει να εμπεδώσετε τα σύνολα μοντέλων.

Στην εργασία αυτή θα πρέπει να συμπληρώσετε κώδικα Python 3 στα σημεία που αναφέρουν # YOUR CODE HERE. Μην τροποποιείτε τον κώδικα που βρίσκεται εκτός αυτών των περιοχών.

Πρωτού παραδόσετε την εργασία σας σιγουρευτείτε ότι ο κώδικας σε όλα τα κελιά τρέχει σωστά. Για το σκοπό αυτό επιλέξτε από το μενού Χρόνος εκτέλεσης (runtime) -> Επανεκίνηση περιόδου λειτουργίας και εκτέλεση όλων.

Συμπληρώστε το όνομα (NAME) και το AEM σας παρακάτω:

In [2]:
NAME = "Kyparissis Kyparissis"
AEM = "10346"

**1** Διαβάστε το διαθέσιμο από την sklearn σύνολο δεδομένων breast cancer, χωρίστε το σε δεδομένα εκπαίδευσης (X_train, y_train) και ελέγχου (X_test, y_test) σε ποσοστό 70%/30% αντίστοιχα με τη συνάρτηση train_test_split (τιμή για random_state βάλτε 0). Το σύνολο αφορά τη διάγνωση καρκίνου του μαστού με βάση μεταβλητές που υπολογίζονται από μια ψηφιοποιημένη εικόνα δείγματος μάζας μαστού που λήφθηκε μέσω αναρρόφησης λεπτής βελόνας (FNA). (2 μονάδες)

In [3]:
# YOUR CODE HERE
# --------------

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

# Load the breast cancer dataset
breast_cancer = load_breast_cancer()

# Split the dataset into training and test sets with a 70% / 30% split
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(breast_cancer.data, breast_cancer.target,
                                                    test_size=test_size,
                                                    random_state=0)

In [4]:
"""Τεστ ορθής ανάγνωσης και διαχωρισμού του συνόλου δεδομένων"""
assert round(X_train[0][8], 5) == 0.1779
assert round(X_test[0][8], 5) == 0.2116

**2** Υλοποιήστε μια ντετερμινιστική εκδοχή της μεθόδου των τυχαίων υποχώρων, η οποία χτίζει τόσα μοντέλα όσες και οι μεταβλητές εισόδου, κάθε ένα από τα οποία αγνοεί και μία διαφορετική μεταβλητή εισόδου. Π.χ. το πρώτο μοντέλο αγνοεί την πρώτη, το δεύτερο αγνοεί τη δεύτερη κτλ. Χρησιμοποιήστε τη συνάρτηση clone από το sklearn.base για να δημιουργήστε αντίγραφο του βασικού μοντέλου σε κάθε επανάληψη. (4 μονάδες)

In [5]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone

class RandomSubspaceDet:
    def __init__(self, estimator=DecisionTreeClassifier()):
        # YOUR CODE HERE
        # --------------

        self.base_estimator = estimator # The base estimator / initial model
        self.models = []                # The models trained on the subsets of the data

    def fit(self, X_train, y_train):
        # YOUR CODE HERE
        # --------------

        self.models = []                # Reset the models for each train process
        n_features = X_train.shape[1]   # Number of features

        # Train a model for each subset of the data (i.e. for each feature removed)
        # and store the model in the models list
        # In loop i, remove the i-th feature from the data
        for i in range(n_features):
            # Remove the i-th feature from the data and get the new data subset
            X_train_subset = np.delete(X_train, i, axis=1)

            # Get a clone of the base estimator and train it on the new data subset
            model = clone(self.base_estimator)
            model.fit(X_train_subset, y_train)

            # Store the model in the models list
            self.models.append(model)

    def predict(self, X):
        # YOUR CODE HERE
        # --------------

        n_samples = X.shape[0]          # Number of samples
        n_models = len(self.models)     # Number of models

        # Store predictions for each sample of X of each model in a 2D array
        predictions = np.zeros((n_samples, n_models))
        # Calculate predictions for each model
        # (Note again, that the model i needs to be trained on the data without the i-th feature)
        for i, model in enumerate(self.models):
            # Get again the data subset without the i-th feature
            X_subset = np.delete(X, i, axis=1)

            # Make predictions for the i-th model using the appropriate data subset (with the i-th feature removed)
            predictions[:, i] = model.predict(X_subset)

        # Majority voting
        # What this does is that for each sample, it counts the number of times each class appears in the predictions
        # and returns the class that appears the most
        # ex. if for a sample, the predictions are [0, 1, 1, 0, 1], the class 1 appears 3 times and the class 0 appears 2 times
        #     -> so the final prediction for this sample will be 1
        majority_vote = lambda x: np.bincount(x.astype(int)).argmax()

        # Apply the weighted majority vote function to each row of the predictions array
        # to get the final predictions of the ensemble for each sample and return them
        final_predictions = np.apply_along_axis(majority_vote,
                                                axis=1,
                                                arr=predictions)
        return final_predictions

In [6]:
"""Τεστ ορθής υλοποίησης RandomSubspaceDet"""
from sklearn.metrics import accuracy_score

rs = RandomSubspaceDet(estimator=DecisionTreeClassifier(random_state=1))
rs.fit(X_train, y_train)
assert round(accuracy_score(rs.predict(X_test), y_test), 4) == 0.9006

**3** Υλοποιήστε τη μέθοδο AdaBoost όπως παρουσιάστηκε στο μάθημα. Χρησιμοποιήστε τη συνάρτηση clone από το sklearn.base για να δημιουργήστε αντίγραφο του βασικού μοντέλου σε κάθε επανάληψη. Χρησιμοποιήστε την παράμετρο sample_weight της fit του βασικού μοντέλου για να ορίσετε τα βάρη των παραδειγμάτων εκπαίδευσης. (4 μονάδες)

In [7]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone

class AdaBoost:
    def __init__(self, n_estimators=20, estimator=DecisionTreeClassifier(max_depth=1)):
        # YOUR CODE HERE
        # --------------

        self.n_estimators = n_estimators    # Number of estimators
        self.base_estimator = estimator     # The base estimator / initial model
        self.estimators = []                # The trained estimators (models) in the ensemble
        self.estimator_weights = []         # The weights of the estimators in the ensemble

    def fit(self, X_train, y_train):
        # YOUR CODE HERE
        # --------------

        self.estimators = []                # Reset the estimators for each train process
        self.estimator_weights = []         # Reset the estimator weights for each train process
        n_samples = X_train.shape[0]        # Number of training samples

        # Initialize all the sample weights to w(i) = 1/N, where N is the number of samples
        sample_weights = np.ones(n_samples) / n_samples

        # Iterate over the number of estimators
        for _ in range(self.n_estimators):
            # Get a clone of the base estimator and train it using the new sample weights
            estimator = clone(self.base_estimator)
            estimator.fit(X_train, y_train,
                          sample_weight=sample_weights)

            # Make predictions using the trained estimator
            prediction = estimator.predict(X_train)
            # Calculate the total error of the estimator using the sample weights
            # The formula is: total_error = sum(w(i) * I(y(i) != h(x(i))))
            total_error = 0
            for i in range(len(sample_weights)):
                if prediction[i] != y_train[i]:
                    total_error += sample_weights[i]

            # If the total error is 0 or greater than 0.5, break the loop
            # If equal to 0, we can't calculate the estimator_weight since we would divide by 0 (precision error)
            # If greater than 0.5, the estimator_weight would be negative
            if total_error == 0 or total_error > 0.5:
                break

            # Determine the weight of the estimator in the ensemble for the final prediction
            estimator_weight = 0.5 * np.log((1 - total_error)/(total_error))

            # Save the estimator and its weight
            self.estimators.append(estimator)
            self.estimator_weights.append(estimator_weight)

            # Update the sample weights using the formula:
            # w(i) = w(i) * exp(estimator_weight) if y(i) != h(x(i))
            # w(i) = w(i) * exp(-estimator_weight) if y(i) == h(x(i))
            for i in range(len(sample_weights)):
                if prediction[i] != y_train[i]:
                    sample_weights[i] *= np.exp(estimator_weight)
                else:
                    sample_weights[i] *= np.exp(-estimator_weight)

            # Normalize the sample weights so that they sum up to 1 (i.e. w(i) = w(i) / sum(w))
            sample_weights = sample_weights / sample_weights.sum()

    def predict(self, X):
        # YOUR CODE HERE
        # --------------

        # Get the predictions of each estimator in the ensemble
        predictions = []
        for estimator in self.estimators:
            predictions.append(estimator.predict(X))

        # Then stack them in a 2D array
        # ex. if we have 3 estimators and 10 samples, the predictions array will have shape (10, 3)
        # where each row will contain the predictions of the 3 estimators for the corresponding sample
        predictions = np.stack(predictions, axis=1)

        # Weighted majority voting
        # For each sample, calculate the weighted majority vote of the predictions of the estimators in the ensemble
        # The weighted majority vote is calculated as follows:
        # For each unique class in the predictions, calculate the sum of the weights of the estimators that predicted this class
        # The class with the highest sum of final weights is the final prediction
        weighted_majority_vote = lambda x: np.unique(x)[np.argmax([np.where(x==categ, self.estimator_weights, 0).sum() for categ in np.unique(x)])]

        # Apply the weighted majority vote function to each row of the predictions array
        # to get the final predictions of the ensemble for each sample and return them
        final_predictions = np.apply_along_axis(weighted_majority_vote,
                                                axis=1,
                                                arr=predictions)
        return final_predictions

In [8]:
"""Τεστ ορθής υλοποίησης AdaBoost"""
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

ab = AdaBoost(n_estimators=20, estimator=DecisionTreeClassifier(max_depth=1, random_state=1))
ab.fit(X_train, y_train)
assert round(accuracy_score(ab.predict(X_test), y_test), 4) == 0.9591


In [9]:
# Ίδιο αποτέλεσμα και με τη κλάση της sklearn
ab = AdaBoostClassifier(n_estimators=20, algorithm="SAMME", estimator=DecisionTreeClassifier(max_depth=1, random_state=1))
ab.fit(X_train, y_train)
assert round(accuracy_score(ab.predict(X_test), y_test), 4) == 0.9591